Library


In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from difflib import SequenceMatcher

Chrome Options

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

Get all link of all club

In [3]:
driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.premierleague.com/clubs'
driver.get(url)
time.sleep(5)
clubs = driver.find_elements(By.CLASS_NAME, 'club-card-wrapper')
name_clubs = []
link_clubs = []

for club in clubs:
    name = club.find_element(By.CLASS_NAME, 'club-card__name').text
    name_clubs.append(name)
    print(name)
    link = club.find_element(By.TAG_NAME, 'a').get_attribute('href')
    link = link.replace("overview", "stats")
    link += "?se=719"
    print(link)
    link_clubs.append(link)

link_all_club = {
    "name_club": name_clubs,
    "link_club": link_clubs,
}

data_frame_link_all_club = pd.DataFrame(link_all_club, index=link_all_club['name_club'])
data_frame_link_all_club.to_csv('../../../Data/Club/link_all_club.csv', index=False)


Arsenal
https://www.premierleague.com/clubs/1/Arsenal/stats?se=719
Aston Villa
https://www.premierleague.com/clubs/2/Aston-Villa/stats?se=719
Bournemouth
https://www.premierleague.com/clubs/127/Bournemouth/stats?se=719
Brentford
https://www.premierleague.com/clubs/130/Brentford/stats?se=719
Brighton & Hove Albion
https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/stats?se=719
Chelsea
https://www.premierleague.com/clubs/4/Chelsea/stats?se=719
Crystal Palace
https://www.premierleague.com/clubs/6/Crystal-Palace/stats?se=719
Everton
https://www.premierleague.com/clubs/7/Everton/stats?se=719
Fulham
https://www.premierleague.com/clubs/34/Fulham/stats?se=719
Ipswich Town
https://www.premierleague.com/clubs/8/Ipswich-Town/stats?se=719
Leicester City
https://www.premierleague.com/clubs/26/Leicester-City/stats?se=719
Liverpool
https://www.premierleague.com/clubs/10/Liverpool/stats?se=719
Manchester City
https://www.premierleague.com/clubs/11/Manchester-City/stats?se=719
Manchester 

Get stats of all club

In [4]:
df_link = pd.read_csv('../../../Data/Club/link_all_club.csv').copy()

df_link = pd.DataFrame(data=list(df_link['link_club']), index=list(df_link['name_club']))

all_data_of_clubs = []

for url in df_link.values:
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url[0])
    time.sleep(5)
    all_data_of_club = driver.find_elements(By.CLASS_NAME, "all-stats__regular-stat-container")
    name = driver.find_element(By.CLASS_NAME, 'club-header__team-name').text
    print(name)
    record = dict()
    for data in all_data_of_club:
        key = data.find_element(By.CLASS_NAME, 'all-stats__regular-stat-name').text
        value = data.find_element(By.CLASS_NAME, 'allStatContainer').text
        if value.__contains__("%"):
            value = value[:-1]
            record[key] = int(value)
        elif value.__contains__("."): 
            record[key] = float(value)
        elif value.__contains__(","):
            value = value.replace(",", "")
            record[key] = int(value)
        else:
            record[key] = int(value)
        
        print(key + ": " + value)
        
    all_data_of_clubs.append(record)
all_data_of_clubs_df = pd.DataFrame(all_data_of_clubs, index=link_all_club['name_club'])

Arsenal
Goals: 28
Goals per match: 2.00
Shots: 201
Shots on target: 74
Shooting accuracy %: 37
Penalties scored: 2
Big Chances Created: 39
Hit woodwork: 2
Passes: 6172
Passes per match: 440.86
Pass accuracy %: 85
Crosses: 257
Cross accuracy %: 23
Clean sheets: 5
Goals Conceded: 14
Goals conceded per match: 1.00
Saves: 39
Tackles: 235
Tackle success %: 62
Blocked shots: 57
Interceptions: 92
Clearances: 258
Headed Clearance: 132
Aerial Battles/Duels Won: 862
Errors leading to goal: 0
Own goals: 0
Yellow cards: 31
Red cards: 3
Fouls: 91
Offsides: 26
Aston Villa
Goals: 23
Goals per match: 1.64
Shots: 194
Shots on target: 64
Shooting accuracy %: 33
Penalties scored: 1
Big Chances Created: 38
Hit woodwork: 4
Passes: 6401
Passes per match: 457.21
Pass accuracy %: 86
Crosses: 241
Cross accuracy %: 25
Clean sheets: 1
Goals Conceded: 23
Goals conceded per match: 1.64
Saves: 36
Tackles: 242
Tackle success %: 60
Blocked shots: 55
Interceptions: 107
Clearances: 241
Headed Clearance: 134
Aerial Batt

Compare two name functions

In [5]:
premier_league_clubs = {
    "Arsenal": "Arsenal",
    "Aston Villa": "Aston Villa",
    "Bournemouth": "AFC Bournemouth",
    "Brentford": "Brentford",
    "Brighton": "Brighton & Hove Albion",
    "Chelsea": "Chelsea",
    "Crystal Palace": "Crystal Palace",
    "Everton": "Everton",
    "Fulham": "Fulham",
    "Ipswich": "Ipswich Town",
    "Leicester": "Leicester City",
    "Liverpool": "Liverpool",
    "Man City": "Manchester City",
    "Man Utd": "Manchester United",
    "Newcastle": "Newcastle United",
    "Nott'm Forest": "Nottingham Forest",
    "Southampton": "Southampton",
    "Spurs": "Tottenham Hotspur",
    "West Ham": "West Ham United",
    "Wolves": "Wolverhampton Wanderers",
}


Get fixtures of all club

In [6]:
import random
all_data_of_clubs_df['Fixtures'] = None
count = -1
    
for url in df_link.values:
    driver = webdriver.Chrome(options=chrome_options)
    url_fixture = url[0].replace("stats?se=719", "fixtures")
    driver.get(url_fixture)
    time.sleep(random.randint(5, 7))
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    elements = driver.find_elements(By.CLASS_NAME, 'fixtures__date-container')
    competitors = []
    count+=1
    for element in elements:
        all_name = element.find_elements(By.CLASS_NAME, 'match-fixture__short-name')
        
        competitor = premier_league_clubs.get(all_name[0].text) if premier_league_clubs.get(all_name[1].text) == link_all_club['name_club'][count] else premier_league_clubs.get(all_name[1].text)
        competitors.append(competitor)
        
    print(link_all_club['name_club'][count])
    print(competitors)
        
    competitors_string = ", ".join(competitors)
    
    all_data_of_clubs_df.loc[link_all_club['name_club'][count], "Fixtures"] = competitors_string

    driver.quit()

Arsenal
['Fulham', 'Everton', 'Crystal Palace', 'Ipswich Town', 'Brentford', 'Brighton & Hove Albion', 'Tottenham Hotspur', 'Aston Villa', 'Wolverhampton Wanderers', 'Manchester City', 'Leicester City', 'West Ham United', 'Nottingham Forest', 'Manchester United', 'Chelsea', 'Fulham', 'Everton', 'Brentford', 'Ipswich Town', 'Crystal Palace', 'AFC Bournemouth', 'Liverpool', 'Newcastle United', 'Southampton']
Aston Villa
['Southampton', 'Nottingham Forest', 'Manchester City', 'Newcastle United', 'Brighton & Hove Albion', 'Leicester City', 'Everton', 'Arsenal', 'West Ham United', 'Wolverhampton Wanderers', 'Ipswich Town', 'Chelsea', 'Crystal Palace', 'Brentford', 'Liverpool', 'Brighton & Hove Albion', 'Nottingham Forest', 'Southampton', 'Newcastle United', 'Manchester City', 'Fulham', 'AFC Bournemouth', 'Tottenham Hotspur', 'Manchester United']
Bournemouth
['AFC Bournemouth', 'West Ham United', 'AFC Bournemouth', 'Crystal Palace', 'AFC Bournemouth', 'Everton', 'AFC Bournemouth', 'AFC Bourn

Get rank of all club

In [7]:
url = 'https://www.premierleague.com/tables'
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
time.sleep(5)

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
    time.sleep(2)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
all_data_of_clubs_df['Rank'] = None
all_data_of_clubs_df['Points'] = None
all_data_of_clubs_df['Performances'] = None
elements = driver.find_elements(By.XPATH, "//*[@class=' ' or @class='tableDark EU_CL' or @class='tableMid EU_CL' or @class='tableMid EU_UC' or @class='tableMid EU_CF']")

for element in elements:
    rank = element.find_element(By.CLASS_NAME, 'league-table__value').text
    name = element.find_element(By.CLASS_NAME, 'league-table__team-name').text
    if name == "Brighton And Hove Albion":
        name = "Brighton & Hove Albion"
    print(name)
    points = element.find_element(By.CLASS_NAME, 'league-table__points').text
    performances_html = element.find_elements(By.CLASS_NAME, 'form-abbreviation')
    performances = []
    for performance in performances_html:
        performances.append(performance.text)
    all_data_of_clubs_df.loc[name, "Rank"] = int(rank)
    all_data_of_clubs_df.loc[name, "Points"] = int(points)
    all_data_of_clubs_df.loc[name, "Performances"] = ", ".join(performances)
    
all_data_of_clubs_df.to_csv("../../../Data/Club/all_data_of_clubs.csv", index=True)

Liverpool
Chelsea
Arsenal
Manchester City
Aston Villa
Brighton & Hove Albion
Brentford
Fulham
Nottingham Forest
Bournemouth
Tottenham Hotspur
Newcastle United
Manchester United
West Ham United
Everton
Crystal Palace
Leicester City
Ipswich Town
Wolverhampton Wanderers
Southampton
